In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data, Dataset
from torch_geometric.nn import GATv2Conv, global_mean_pool, global_max_pool

# --- CONFIGURAÇÃO ---
# Aponta para a pasta onde geraste os .npz no passo anterior
GRAPHS_DIR = 'graphs_npz'
DATASET_CSV = 'dataset_molecular_gnn_ready.csv'

BATCH_SIZE = 32
LEARNING_RATE = 0.0005
EPOCHS = 25
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"Configuração definida. A usar device: {DEVICE}")

Configuração definida. A usar device: cpu


In [3]:
class AntibodyGraphDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.df = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

        # Mapeamento de Aminoácidos para Inteiros (para reconstruir features se faltarem no npz)
        self.aa_map = {k: v for v, k in enumerate("ACDEFGHIKLMNPQRSTVWY")}
        self.aa_map['X'] = 20 # Desconhecido

        # Preparar Encoder de Variantes
        self.label_encoder = LabelEncoder()
        self.df['variant_encoded'] = self.label_encoder.fit_transform(self.df['variant_target'])
        self.num_variants = len(self.label_encoder.classes_)

        print(f"Dataset inicializado. {len(self.df)} amostras.")
        print(f"Variantes únicas: {self.num_variants}")

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # 1. Identificar o ficheiro
        row = self.df.iloc[idx]
        pdb_id = str(row['pdb_id'])
        npz_path = os.path.join(self.root_dir, f"{pdb_id}.npz")

        # 2. Carregar o NPZ
        if not os.path.exists(npz_path):
            # Fallback seguro se faltar ficheiro (evita crashar treino)
            return self.__getitem__((idx + 1) % len(self.df))

        try:
            data_npz = np.load(npz_path)

            # 3. Construir Tensores
            # Coordenadas
            pos = torch.from_numpy(data_npz['coords']).float()

            # Arestas
            edge_index = torch.from_numpy(data_npz['edge_index']).long()

            # Features dos Nós (x)
            # Tenta carregar do NPZ, se não existir, reconstrói da sequência no CSV
            if 'x' in data_npz:
                x = torch.from_numpy(data_npz['x']).long()
            else:
                x = self._reconstruct_node_features(row, len(pos))

            # Labels e Variante
            y = torch.tensor([row['label']], dtype=torch.float)
            variant_id = torch.tensor([row['variant_encoded']], dtype=torch.long)

            # Objeto Data PyG
            data = Data(x=x, edge_index=edge_index, pos=pos, y=y)
            data.variant_id = variant_id # Anexar ID da variante

            return data

        except Exception as e:
            print(f"Erro ao carregar {pdb_id}: {e}")
            return self.__getitem__((idx + 1) % len(self.df))

    def _reconstruct_node_features(self, row, num_nodes_graph):
        """
        Reconstrói features dos nós a partir da sequência de texto (H + L)
        se elas não estiverem no ficheiro .npz.
        """
        # Concatena cadeias H e L (assumindo que foi esta a ordem de extração)
        full_seq = (str(row['chain_heavy']) + str(row['chain_light'])).upper()

        # Se os tamanhos não baterem certo (comum em PDBs com resíduos em falta),
        # criamos features dummy para não crashar, mas idealmente corrigia-se o pré-processamento.
        if len(full_seq) != num_nodes_graph:
            # Fallback: features aleatórias ou zeros (apenas para não parar o código)
            # Num cenário real, deve-se garantir a consistência no passo anterior.
            indices = [0] * num_nodes_graph
        else:
            indices = [self.aa_map.get(aa, 20) for aa in full_seq]

        return torch.tensor(indices, dtype=torch.long).unsqueeze(1)

In [4]:
class HybridNeutralizationModel(nn.Module):
    """
    Arquitetura Híbrida (GNN + Semântica) atualizada para o Módulo 1.
    """
    def __init__(self, num_node_features=1, num_variants=10, embedding_dim=64, hidden_dim=128):
        super(HybridNeutralizationModel, self).__init__()

        # --- GNN (Estrutura) ---
        self.node_embedding = nn.Embedding(21, hidden_dim) # 20 aminoácidos + 1 desconhecido

        self.conv1 = GATv2Conv(hidden_dim, hidden_dim, heads=4, concat=True, dropout=0.1)
        self.conv2 = GATv2Conv(hidden_dim * 4, hidden_dim, heads=1, concat=False, dropout=0.1)

        # --- NLP (Variante) ---
        self.variant_embedding = nn.Embedding(num_embeddings=num_variants, embedding_dim=hidden_dim)

        # --- Fusão ---
        self.fusion_dim = hidden_dim * 2

        self.classifier = nn.Sequential(
            nn.Linear(self.fusion_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, data, variant_ids):
        # 1. GNN
        x, edge_index, batch = data.x, data.edge_index, data.batch

        # Garantir que x é LongTensor para o Embedding
        x = x.long()
        if x.dim() > 1: x = x.squeeze()

        x = self.node_embedding(x)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = self.conv2(x, edge_index)
        x = F.elu(x)

        # Global Pooling (Nós -> Grafo)
        x_graph = global_mean_pool(x, batch) + global_max_pool(x, batch)

        # 2. Variante
        x_variant = self.variant_embedding(variant_ids)

        # 3. Fusão
        combined = torch.cat([x_graph, x_variant], dim=1)

        # 4. Classificação
        logits = self.classifier(combined)
        return logits

# Especificação Técnica da Arquitetura de Modelação (Módulo 1)

## 1. Definição Formal do Modelo

O modelo desenvolvido é classificado como uma **Rede Neuronal Híbrida Multi-Modal (*Multi-Modal Hybrid Neural Network*)**.

A sua função matemática, , procura aproximar a probabilidade condicional de neutralização , dados dois inputs heterogéneos: a estrutura topológica do anticorpo () e a identidade semântica da variante viral ().

Onde:

*  é o grafo molecular do anticorpo.
*  é o identificador categórico da variante.
*  representa os parâmetros treináveis da rede.
*  é a função de ativação Sigmóide que mapeia o output para o intervalo .

---

## 2. Decomposição da Arquitetura

A rede está estruturada em três blocos funcionais sequenciais:

### Bloco A: Codificador Estrutural (The Structural Encoder)

**Objetivo:** Transformar a estrutura 3D complexa e esparsa do anticorpo num vetor de características latentes de dimensão fixa.

* **Representação de Entrada (Input):** O anticorpo não é tratado como uma imagem (grid Euclidiana), mas sim como um **Grafo Geométrico não-Euclidiano**.
* **Nós ():** Cada nó representa um resíduo de aminoácido (Cadeias Pesada e Leve).
* **Arestas ():** Definidas por um *cutoff* de distância espacial (< 10 Ångströms) entre Carbonos-Alfa (). Isto captura as interações não-covalentes e a geometria de dobragem da proteína.
* **Features dos Nós:** Inicialmente, cada nó possui apenas um índice inteiro representando o seu tipo de aminoácido (0-20). Uma camada de *Embedding* () projeta este índice num espaço vetorial denso (), permitindo à rede aprender propriedades físico-químicas (ex: hidrofobicidade, carga) de forma autónoma.


* **Mecanismo de Processamento: Graph Attention Networks (GATv2):**
Em vez de convoluções em grafos padrão (GCN), utilizamos a arquitetura **GATv2**.
* *Justificação Teórica:* Numa interação anticorpo-antigénio, nem todos os resíduos são iguais. Apenas os resíduos nas regiões CDR (*Complementarity-Determining Regions*) interagem diretamente com o vírus.
* *Mecanismo:* A GATv2 aplica um **Mecanismo de Atenção (*Self-Attention*)**. Para cada nó, a rede calcula uma pontuação de importância para os seus vizinhos. Isto permite ao modelo "aprender a olhar" preferencialmente para as regiões estruturais críticas para a ligação, ignorando o "ruído" do esqueleto proteico conservado.


* **Agregação (Global Pooling):**
Após as camadas convolucionais, o grafo de  nós precisa de ser reduzido a um único vetor que represente a molécula inteira. Utilizamos uma estratégia híbrida:



Isto captura tanto a composição média do anticorpo como a presença de características locais fortes (motivos estruturais específicos).

### Bloco B: Codificador Semântico (The Semantic Encoder)

**Objetivo:** Injetar o contexto biológico da variante viral alvo.

* **O Problema dos Dados:** Como o dataset de treino possui apenas a identificação nominal das variantes (ex: "Omicron", "Delta") e não as suas estruturas 3D complexadas, a estrutura viral não pode ser inserida na GNN.
* **A Solução (Learned Embeddings):**
Utilizamos uma camada de *Embedding* que mapeia cada ID de variante () para um vetor denso ().
* Durante o processo de *Backpropagation*, a rede ajusta os valores deste vetor.
* **Interpretação:** Variantes que são neutralizadas pelos mesmos anticorpos acabarão por ter representações vetoriais matematicamente próximas neste espaço latente ("Clusterização Funcional").
Justificação da Escolha:
* **Roadmap Futuro**: Para a versão final, planeamos enriquecer o dataset com sequências externas (NCBI) para substituir este módulo por um processador de sequências (ProtBERT), permitindo generalização total.



### Bloco C: Módulo de Fusão e Classificação (Fusion & MLP Head)

**Objetivo:** Correlacionar a estrutura do anticorpo com a identidade do vírus para prever o fenótipo.

* **Fusão Multi-Modal:** Os vetores latentes do anticorpo () e da variante () são concatenados, criando um vetor único que representa o **par biológico**.


* **Perceptrão Multicamada (MLP):**
O vetor combinado atravessa camadas densas (*Linear Layers*) que aprendem as relações não-lineares complexas entre a forma do anticorpo e o tipo de vírus.
* **Regularização e Estabilidade:**
* **Batch Normalization:** Normaliza os outputs de cada camada intermédia para estabilizar e acelerar o treino.
* **Dropout (0.3):** Desativa aleatoriamente 30% dos neurónios durante o treino. Isto introduz ruído estocástico que previne o modelo de "memorizar" exemplos específicos (*Overfitting*), forçando-o a aprender padrões generalizáveis.



---

## 3. Fluxo de Aprendizagem (Training Dynamics)

Embora a classe `HybridNeutralizationModel` defina a estrutura (o "corpo"), a inteligência surge através do processo de otimização:

1. **Forward Pass (Inferência):** Os dados fluem da entrada para a saída, gerando uma previsão .
2. **Cálculo de Perda (Loss Function):** Utilizamos a **Entropia Cruzada Binária com Logits (*BCEWithLogitsLoss*)**. Esta função mede a distância matemática entre a previsão do modelo e a realidade biológica (0 ou 1).
3. **Backward Pass (Retropropagação):** O algoritmo de *Backpropagation* calcula o gradiente do erro em relação a cada um dos ~315.000 parâmetros da rede.
4. **Otimização (Adam):** O otimizador atualiza os pesos na direção oposta ao gradiente para minimizar o erro na próxima iteração.

---

## 4. Resumo das Inovações (Módulo 1)

Esta arquitetura cumpre os requisitos de inovação do projeto através de:

1. **Deep Learning Geométrico:** Abandono de descritores lineares clássicos em favor de representações baseadas em grafos que preservam a topologia 3D nativa.
2. **Mecanismos de Atenção:** Uso de GATv2 para mimetizar o foco biológico nas regiões de interação.